In [1]:
import music21
import pickle
import numpy as np
from music21 import duration

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2023-04-27 20:51:34.257028: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 20:51:37.610128: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  1


2023-04-27 20:51:39.379682: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-27 20:51:39.793952: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-27 20:51:39.794011: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


### Load data from files

https://verovio.humdrum.org/?file=essen/europa/deutschl/altdeu2/deut4207.krn

In [3]:
data = []

In [4]:
files = ["data_germany_1","data_germany_2","data_germany_3","data_germany_4","data_germany_5","data_germany_6","data_germany_7", "data_germany_8", "data_germany_9", "data_germany_10", "data_germany_11"]
for file in files:
    with open('data/data/{}.pkl'.format(file), 'rb') as handle:
        data.extend(pickle.load(handle))

### Extracting the data

In [5]:
notes_input = []
durations_input = []
phrases_input = []

for piece in data:
    notes = piece[0]
    notes_input.append(129) # START = 129
    notes_input.extend(notes)

    durations = piece[1]
    durations_input.append(0)
    durations_input.extend(durations)

    phrases = piece[2]
    phrases_input.append(0)
    phrases_input.extend(phrases)

In [6]:
print("Length of the notes_input: {}".format(len(notes_input)))
print("Twenty first elements: {}".format(notes_input[:20]))
print("Length of the durations_input: {}".format(len(durations_input)))
print("Twenty first elements: {}".format(durations_input[:20]))
print("Length of the phrases_input: {}".format(len(phrases_input)))
print("Twenty first elements: {}".format(phrases_input[:20]))

Length of the notes_input: 282505
Twenty first elements: [129, 55, 60, 60, 60, 60, 60, 62, 64, 64, 64, 67, 62, 128, 67, 60, 128, 55, 60, 60]
Length of the durations_input: 282505
Twenty first elements: [0, 0.5, 0.5, 0.5, 0.375, 0.125, 0.25, 0.25, 0.5, 0.5, 0.5, 0.5, 1.0, 0.5, 0.5, 1.0, 0.5, 0.5, 0.5, 0.5]
Length of the phrases_input: 282505
Twenty first elements: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]


In [7]:
durations_to_int = {}
int_to_duration = {}
notes_to_int = {}
int_to_notes = {}
phrases_to_int = {}
int_to_phrases = {}
for index, duration in enumerate(sorted(set(durations_input))):
    durations_to_int[duration] = index
    int_to_duration[index] = duration

for index, note in enumerate(sorted(set(notes_input))):
    notes_to_int[note] = index
    int_to_notes[index] = note

for index, phrase in enumerate(sorted(set(phrases_input))):
    phrases_to_int[phrase] = index
    int_to_phrases[index] = phrase

durations_network_input = [ durations_to_int[duration] for duration in durations_input ]
notes_network_input = [ notes_to_int[note] for note in notes_input ]
phrases_network_input = [ phrases_to_int[phrase] for phrase in phrases_input ]

In [8]:
int_to_duration

{0: 0,
 1: 0.00390625,
 2: 0.125,
 3: Fraction(1, 6),
 4: 0.25,
 5: Fraction(1, 3),
 6: 0.375,
 7: 0.5,
 8: Fraction(2, 3),
 9: 0.75,
 10: 1.0,
 11: Fraction(4, 3),
 12: 1.5,
 13: 2.0,
 14: 3.0,
 15: 4.0,
 16: 6.0,
 17: 8.0,
 18: 12.0}

In [9]:
print("Length of the notes_input: {}".format(len(notes_network_input)))
print("Twenty first elements: {}".format(notes_network_input[:20]))
print("Length of the durations_input: {}".format(len(durations_network_input)))
print("Twenty first elements: {}".format(durations_network_input[:20]))
print("Length of the phrases_input: {}".format(len(phrases_network_input)))
print("Twenty first elements: {}".format(phrases_network_input[:20]))

Length of the notes_input: 282505
Twenty first elements: [45, 10, 15, 15, 15, 15, 15, 17, 19, 19, 19, 22, 17, 44, 22, 15, 44, 10, 15, 15]
Length of the durations_input: 282505
Twenty first elements: [0, 7, 7, 7, 6, 2, 4, 4, 7, 7, 7, 7, 10, 7, 7, 10, 7, 7, 7, 7]
Length of the phrases_input: 282505
Twenty first elements: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]


In [10]:
n_notes = len(notes_to_int)
n_durations = len(durations_to_int)
n_phrases = len(phrases_to_int)

In [11]:
print("Number of distinct notes: {}".format(n_notes))
print("Number of distinct durations: {}".format(n_durations))

Number of distinct notes: 46
Number of distinct durations: 19


In [12]:
with open('dictionary.pkl', 'wb') as file:
    dictionary = { "durations_to_int" : durations_to_int, "int_to_duration" : int_to_duration, "notes_to_int" : notes_to_int, "int_to_notes" : int_to_notes, "phrases_to_int" : phrases_to_int, "int_to_phrases" : int_to_phrases }
    # A new file will be created
    pickle.dump(dictionary, file)

In [13]:
import os
import pickle
import numpy
from music21 import note, chord

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import plot_model
from models.RNNAttentionApproach2 import prepare_sequences, create_network

In [14]:
seq_len = 32
network_input, network_output = prepare_sequences(notes_network_input, durations_network_input, phrases_network_input, seq_len)

In [15]:
print('note input')
print(network_input[0][0])
print('duration input')
print(network_input[1][0])
print('phrases_output')
print(network_output[0][0])

note input
[45 10 15 15 15 15 15 17 19 19 19 22 17 44 22 15 44 10 15 15 15 15 17 19
 19 19 22 17 44 22 15 44]
duration input
[ 0  7  7  7  6  2  4  4  7  7  7  7 10  7  7 10  7  7  7  7  7  4  4  7
  7  7  7 10  7  7 10  7]
phrases_output
[1. 0.]


In [16]:
embed_size = 100
rnn_units = 256
use_attention = True
model, att_model = create_network(n_notes, n_durations, n_phrases, embed_size, rnn_units, use_attention)
model.summary()

2023-04-27 20:51:46.600514: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-27 20:51:46.600956: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-27 20:51:46.601027: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-27 20:51:48.546941: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-27 20:51:48.547098: I tensorflow/compile

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 100)    4600        ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 100)    1900        ['input_2[0][0]']                
                                                                                              

2023-04-27 20:51:49.229282: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-27 20:51:49.231832: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-27 20:51:49.234503: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [17]:
import numpy as np
weights_folder = "weights" #os.path.join(run_folder, 'weights')
# model.load_weights(os.path.join(weights_folder, "weights.h5"))


checkpoint1 = ModelCheckpoint(
    os.path.join(weights_folder, "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

checkpoint2 = ModelCheckpoint(
    os.path.join(weights_folder, "weights.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss'
    , restore_best_weights=True
    , patience = 10
)


callbacks_list = [
    checkpoint1
    , checkpoint2
    , early_stopping
 ]

model.save_weights(os.path.join(weights_folder, "weights.h5"))
model.fit(network_input, network_output
          , epochs=2000000, batch_size=32
          , validation_split = 0.2
          , callbacks=callbacks_list
          , shuffle=True
         )

2023-04-27 20:51:49.518110: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 57850368 exceeds 10% of free system memory.
2023-04-27 20:51:49.670037: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 57850368 exceeds 10% of free system memory.
2023-04-27 20:51:49.848978: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 57850368 exceeds 10% of free system memory.
2023-04-27 20:51:49.959434: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 57850368 exceeds 10% of free system memory.


Epoch 1/2000000


2023-04-27 20:51:50.936909: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-27 20:51:50.949710: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-27 20:51:50.954978: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

7062/7062 [==============================] - ETA: 0s - loss: 0.3529

2023-04-27 20:56:13.816366: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-27 20:56:13.818813: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-27 20:56:13.820447: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

7062/7062 [==============================] - 289s 36ms/step - loss: 0.3529 - val_loss: 0.3381
Epoch 2/2000000
7062/7062 [==============================] - 238s 34ms/step - loss: 0.3159 - val_loss: 0.2997
Epoch 3/2000000
7062/7062 [==============================] - 196s 28ms/step - loss: 0.2906 - val_loss: 0.2753
Epoch 4/2000000
7062/7062 [==============================] - 197s 28ms/step - loss: 0.2737 - val_loss: 0.2633
Epoch 5/2000000
7062/7062 [==============================] - 199s 28ms/step - loss: 0.2549 - val_loss: 0.2561
Epoch 6/2000000
7062/7062 [==============================] - 211s 30ms/step - loss: 0.2446 - val_loss: 0.2492
Epoch 7/2000000
7062/7062 [==============================] - 203s 29ms/step - loss: 0.2372 - val_loss: 0.2413
Epoch 8/2000000
7062/7062 [==============================] - 221s 31ms/step - loss: 0.2320 - val_loss: 0.2482
Epoch 9/2000000
7062/7062 [==============================] - 206s 29ms/step - loss: 0.2298 - val_loss: 0.2547
Epoch 10/2000000
7062/7062